# Dataset 1 - Artificial Neural Network - Tyler Babcock

In [1]:
# Code adapted from discussions and hw3 demo
# Changed so it would read our data and drop the
# correct class. One hot encoding split class differently.
# Changed hyperparameter values tested.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from IPython.display import display
from sklearn.svm import OneClassSVM
from numpy import where

In [2]:
df = pd.read_csv('train.csv') # Data set 1 Training Data
print(df.head())

   battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  m_dep  \
0            842     0          2.2         0   1       0           7    0.6   
1           1021     1          0.5         1   0       1          53    0.7   
2            563     1          0.5         1   2       1          41    0.9   
3            615     1          2.5         0   0       0          10    0.8   
4           1821     1          1.2         0  13       1          44    0.6   

   mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  talk_time  \
0        188        2  ...         20       756  2549     9     7         19   
1        136        3  ...        905      1988  2631    17     3          7   
2        145        5  ...       1263      1716  2603    11     2          9   
3        131        6  ...       1216      1786  2769    16     8         11   
4        141        2  ...       1208      1212  1411     8     2         15   

   three_g  touch_screen  wifi  price_

## Scaling

In [3]:
X = df.drop('price_range', axis = 1)
y = df['price_range']

# normalize data so every feature has the same scale
scaler = MinMaxScaler(feature_range=(0, 1))
X_rescaled = scaler.fit_transform(X)
X = pd.DataFrame(data = X_rescaled, columns = X.columns)

# One hot encoding
set_of_classes = y.value_counts().index.tolist()
set_of_classes= pd.DataFrame({'species': set_of_classes})
y = pd.get_dummies(y)

print("Pre-processed data :")
print(X.head())

print("Pre-processed class :")
print(y.head())

print("Dimensions of the dataset : ", df.shape)

Pre-processed data :
   battery_power  blue  clock_speed  dual_sim        fc  four_g  int_memory  \
0       0.227789   0.0         0.68       0.0  0.052632     0.0    0.080645   
1       0.347361   1.0         0.00       1.0  0.000000     1.0    0.822581   
2       0.041416   1.0         0.00       1.0  0.105263     1.0    0.629032   
3       0.076152   1.0         0.80       0.0  0.000000     0.0    0.129032   
4       0.881764   1.0         0.28       0.0  0.684211     1.0    0.677419   

      m_dep  mobile_wt   n_cores    pc  px_height  px_width       ram  \
0  0.555556   0.900000  0.142857  0.10   0.010204  0.170895  0.612774   
1  0.666667   0.466667  0.285714  0.30   0.461735  0.993324  0.634687   
2  0.888889   0.541667  0.571429  0.30   0.644388  0.811749  0.627205   
3  0.777778   0.425000  0.714286  0.45   0.620408  0.858478  0.671566   
4  0.555556   0.508333  0.142857  0.70   0.616327  0.475300  0.308658   

       sc_h      sc_w  talk_time  three_g  touch_screen  wifi  
0

## Model Training

In [4]:
data_train, data_test, class_train, class_test = train_test_split(X, y, test_size=0.2)

# (# Hidden Layers, # Nodes per Layer) = (1, 1)
# Learning rate is 0.1
# Number of epochs is 1000
mlp = MLPClassifier(solver = 'sgd', random_state = 42, activation = 'logistic', 
                    learning_rate_init = 0.05, batch_size = 100, 
                    hidden_layer_sizes = (10, 12), max_iter = 400)

y = pd.get_dummies(y)
mlp.fit(data_train, class_train)

pred = mlp.predict(data_test)

print("Accuracy : ", accuracy_score(class_test, pred))
print("Mean Square Error : ", mean_squared_error(class_test, pred))

print(pred[:5])
print("Confusion Matrix for each label : ")
print(multilabel_confusion_matrix(class_test, pred))

print("Classification Report : ")
print(classification_report(class_test, pred))

Accuracy :  0.9525
Mean Square Error :  0.023125
[[0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]]
Confusion Matrix for each label : 
[[[296   2]
  [  5  97]]

 [[286  10]
  [  6  98]]

 [[292   6]
  [  5  97]]

 [[308   0]
  [  3  89]]]
Classification Report : 
              precision    recall  f1-score   support

           0       0.98      0.95      0.97       102
           1       0.91      0.94      0.92       104
           2       0.94      0.95      0.95       102
           3       1.00      0.97      0.98        92

   micro avg       0.95      0.95      0.95       400
   macro avg       0.96      0.95      0.95       400
weighted avg       0.96      0.95      0.95       400
 samples avg       0.95      0.95      0.95       400



C:\Users\Tyler\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Cross Validation

In [5]:
CV = cross_validate(mlp, X, y, cv=8, scoring=['accuracy', 'neg_mean_squared_error'])
print('Accuracy')
print(CV['test_accuracy'])
print('MSE')
print(-1*CV['test_neg_mean_squared_error'])

print('Average Accuracy = ', sum(CV['test_accuracy']) / len(CV['test_accuracy']))
print('Average MSE = ', sum(-1 * CV['test_neg_mean_squared_error']) / len(CV['test_neg_mean_squared_error']))

C:\Users\Tyler\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy
[0.944 0.96  0.94  0.948 0.944 0.964 0.952 0.952]
MSE
[0.026 0.02  0.029 0.026 0.027 0.018 0.024 0.023]
Average Accuracy =  0.9504999999999999
Average MSE =  0.024125


## Hyperparameter Tuning

In [6]:
max_iterations = [400, 600, 800]
hidden_layer_siz = [(6, 8), (7, 9), (10, 12)]
learning_rates = 0.05 * np.arange(1, 6)

param_grid = dict(learning_rate_init = learning_rates, hidden_layer_sizes = hidden_layer_siz, max_iter = max_iterations)
# set model
mlp = MLPClassifier(solver = 'sgd', random_state = 42, activation = 'logistic', learning_rate_init = 0.3, batch_size = 100, hidden_layer_sizes = (12, 3), max_iter = 500)

# For Grid Search
grid = GridSearchCV(estimator = mlp, param_grid = param_grid)

# For Random Search
# grid = RandomizedSearchCV(estimator = mlp, param_distributions = param_grid, n_iter = 10)

grid.fit(X,y)

GridSearchCV(estimator=MLPClassifier(activation='logistic', batch_size=100,
                                     hidden_layer_sizes=(12, 3),
                                     learning_rate_init=0.3, max_iter=500,
                                     random_state=42, solver='sgd'),
             param_grid={'hidden_layer_sizes': [(6, 8), (7, 9), (10, 12)],
                         'learning_rate_init': array([0.05, 0.1 , 0.15, 0.2 , 0.25]),
                         'max_iter': [400, 600, 800]})

In [7]:
print("Optimal Hyper-parameters : ", grid.best_params_)
print("Optimal Accuracy : ", grid.best_score_)

Optimal Hyper-parameters :  {'hidden_layer_sizes': (10, 12), 'learning_rate_init': 0.05, 'max_iter': 400}
Optimal Accuracy :  0.959
